# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,24.46,94,100,1.54,US,1691272499
1,1,papatowai,-46.5619,169.4708,8.19,73,100,3.20,NZ,1691272499
2,2,vorgashor,67.5833,63.9500,17.26,55,0,3.08,RU,1691272499
3,3,puerto natales,-51.7236,-72.4875,5.28,88,100,11.45,CL,1691272499
4,4,adamstown,-25.0660,-130.1015,19.97,67,23,7.55,PN,1691272499


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    frame_width = 700,
    frame_height = 500,
    tiles='OSM',
    size = 'Humidity',
    geo = True,
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_df = (city_data_df["Max Temp"]<27)& (city_data_df["Max Temp"]>21)
wind_df = (city_data_df["Wind Speed"]<4.5)
cloudiness_df = (city_data_df["Cloudiness"]==0)
# Drop any rows with null values
reduced_df = city_data_df.loc[(city_data_df["Max Temp"]<27)& (city_data_df["Max Temp"]>=21) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0)]
reduced_df = reduced_df.dropna()


# Display sample data
reduced_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,taroudant,30.4703,-8.8770,21.65,73,0,3.08,MA,1691272505
87,87,mordovo,52.0834,40.7700,22.70,67,0,2.42,RU,1691272509
91,91,toliara,-23.3500,43.6667,21.25,58,0,3.69,MG,1691272509
118,118,gravenhurst,44.9168,-79.3663,24.10,35,0,2.06,CA,1691272511
145,145,fort mcmurray,56.7268,-111.3810,24.77,50,0,3.60,CA,1691272514
147,147,sao vicente,-23.9631,-46.3919,25.10,51,0,2.06,BR,1691272514
150,150,mil'kovo,55.6078,37.7956,22.81,71,0,2.02,RU,1691272514
161,161,horqueta,-23.3428,-57.0597,26.59,31,0,1.35,PY,1691272515
209,209,jardim,-21.4803,-56.1381,26.31,28,0,1.86,BR,1691272276
371,371,ketchikan,55.3422,-131.6461,24.26,53,0,3.60,US,1691272535


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame()
hotel_df = reduced_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
47,taroudant,MA,30.4703,-8.8770,73,
87,mordovo,RU,52.0834,40.7700,67,
91,toliara,MG,-23.3500,43.6667,58,
118,gravenhurst,CA,44.9168,-79.3663,35,
145,fort mcmurray,CA,56.7268,-111.3810,50,
147,sao vicente,BR,-23.9631,-46.3919,51,
150,mil'kovo,RU,55.6078,37.7956,71,
161,horqueta,PY,-23.3428,-57.0597,31,
209,jardim,BR,-21.4803,-56.1381,28,
371,ketchikan,US,55.3422,-131.6461,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
            "limit": "1",
            "apiKey": geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address =  name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taroudant - nearest hotel: Hotel Tiouit
mordovo - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
gravenhurst - nearest hotel: Howard-Johnson
fort mcmurray - nearest hotel: Nomad Hotel and Suites
sao vicente - nearest hotel: Pousada Vitória
mil'kovo - nearest hotel: Дрозды
horqueta - nearest hotel: No hotel found
jardim - nearest hotel: Hotel Uchôa
ketchikan - nearest hotel: Cape Fox Lodge
peymeinade - nearest hotel: Hôtel de la Poste
capitan bado - nearest hotel: No hotel found
frenda - nearest hotel: No hotel found
pyetrykaw - nearest hotel: No hotel found
serdobsk - nearest hotel: "Сердобск"
saquarema - nearest hotel: Pousada Garota de Itaúna
basyun - nearest hotel: No hotel found
huzhou - nearest hotel: 浙北大酒店
armacao de buzios - nearest hotel: Vila da Santa
mweka - nearest hotel: No hotel found
beclean - nearest hotel: No hotel found
dachne - nearest hotel: No hotel found
oujda-angad - nearest hotel: No hotel found
paphos - nearest hotel: Mayfair

,City,Country,Lat,Lng,Humidity,Hotel Name
47,taroudant,MA,30.4703,-8.8770,73,Hotel Tiouit
87,mordovo,RU,52.0834,40.7700,67,No hotel found
91,toliara,MG,-23.3500,43.6667,58,Ambary
118,gravenhurst,CA,44.9168,-79.3663,35,Howard-Johnson
145,fort mcmurray,CA,56.7268,-111.3810,50,Nomad Hotel and Suites
147,sao vicente,BR,-23.9631,-46.3919,51,Pousada Vitória
150,mil'kovo,RU,55.6078,37.7956,71,Дрозды
161,horqueta,PY,-23.3428,-57.0597,31,No hotel found
209,jardim,BR,-21.4803,-56.1381,28,Hotel Uchôa
371,ketchikan,US,55.3422,-131.6461,53,Cape Fox Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    frame_width = 700,
    frame_height = 500,
    tiles='OSM',
    size = 'Humidity',
    geo = True,
    scale = 1,
    color = "City",
    hover_cols = [ 'Country','Hotel Name'],
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)